In [1]:
import sys

sys.path.insert(0, '..')

from app.processing.audio.SIWhisperModel import SIWhisperModel
from app.SIWeaviateClient import SIWeaviateClient
from app.processing.YoutubeProcessor import YoutubeProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/google/api_core/exceptions.py:37: ImportWarning: Please install grpcio-status to obtain helpful grpc error messages.
  warnings.warn(
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
with SIWeaviateClient() as client:
    YoutubeProcessor(client=client, whisper=whisper, youtube_url="https://www.youtube.com/watch?v=fdlg_TVM868")

AUDIO CODEC aac


whisper-medium: 100%|██████████| 295.497/295.497 [00:19<00:00, 15.10s/s]           


time_taken 20.011574506759644
